In [1]:
suppressPackageStartupMessages({
    library(dplyr)
    library(SingleCellExperiment)
    library(miloR)
    library(Matrix)
    # library(scater)
    # library(scran)
    library(patchwork)
})

# Old

In [2]:
h5ad="/scratch/peidli/pertpy_benchmark/milo_data_1000.h5ad"
obs="/scratch/peidli/pertpy_benchmark/milo_obs_1000.csv"
obsm_scvi="/scratch/peidli/pertpy_benchmark/milo_obsm_scvi_1000.csv"
obsm_umap="/scratch/peidli/pertpy_benchmark/milo_obsm_umap_1000.csv"
mtx="/scratch/peidli/pertpy_benchmark/milo_data_1000.mtx"

# Construct SCE object with empry counts and logcounts
obs <- read.csv(obs)
obsm_scvi <- read.csv(obsm_scvi)
obsm_umap <- read.csv(obsm_umap)
mtx <-Matrix::readMM(mtx)
# transpose the matrix
mtx <- t(mtx)

In [7]:
sce <- SingleCellExperiment(
    assays = list(
        counts = mtx,
        logcounts = mtx
    ),
    colData = obs,
    reducedDims = list(
        scvi = as.matrix(obsm_scvi),
        umap = as.matrix(obsm_umap)
    )
)

In [4]:
sce

class: SingleCellExperiment 
dim: 16299 1000 
metadata(0):
assays(2): counts logcounts
rownames: NULL
rowData names(0):
colnames: NULL
colData names(22): X Collection_Day ... cell_type dataset_group
reducedDimNames(2): scvi umap
mainExpName: NULL
altExpNames(0):

In [5]:
dim(mtx)
dim(obs)        # Number of rows should equal number of cells
dim(reducedDim(sce, "scvi"))
dim(reducedDim(sce, "umap"))

[1] 16299  1000

[1] 1000   22

[1] 1000   11

[1] 1000    3

In [8]:
# Set a seed for reproducibility
set.seed(69)

# Assign random unique row names (e.g., gene identifiers)
rownames(sce) <- paste0("gene_", sample(1e7, nrow(sce)))

# Assign random unique column names (e.g., cell identifiers)
colnames(sce) <- paste0("cell_", sample(1e7, ncol(sce)))

In [9]:
milo.obj <- Milo(sce)

In [ ]:
milo.obj <- buildGraph(milo.obj, k=15, d=10, reduced.dim="scvi")

Warning message in storage.mode(X) <- "double":
“NAs introduced by coercion”


In [ ]:

milo.obj <- makeNhoods(milo.obj, k=15, d=10, refined=TRUE, prop=0.01, refinement_scheme="graph")

Warning message in storage.mode(X) <- "double":
“NAs introduced by coercion”


# Other data

In [3]:
data("sim_trajectory", package = "miloR")

In [44]:
## Extract SingleCellExperiment object
traj_sce <- sim_trajectory[['SCE']]
colData(traj_sce) <- DataFrame(sim_trajectory[["meta"]])
colnames(traj_sce) <- colData(traj_sce)$cell_id
redim <- reducedDim(traj_sce, "PCA")
dimnames(redim) <- list(colnames(traj_sce), paste0("PC", c(1:50)))
reducedDim(traj_sce, "PCA") <- redim 

In [19]:
# Suppose sce is your SingleCellExperiment object.
# Define the desired new total number of cells.
new_total_cells <- 2000  # for example, doubling from 1000 to 2000

# Sample column indices with replacement.
new_indices <- sample(seq_len(ncol(traj_sce)), size = new_total_cells, replace = TRUE)

# Create a new SCE with upsampled cells.
traj_sce_upsampled <- traj_sce[, new_indices]

Constructing kNN graph with k:150



In [24]:
traj_milo <- buildGraph(traj_milo, k=150, d=10, reduced.dim="PCA")
traj_milo <- makeNhoods(traj_milo, k=15, d=10, refined=TRUE, prop=0.01, refinement_scheme="graph")
traj_milo <- countCells(traj_milo, meta.data = data.frame(colData(traj_milo)), samples="Sample")

Checking valid object

Running refined sampling with reduced_dim



Checking valid object

Running refined sampling with graph



Checking meta.data validity

Counting cells in neighbourhoods



In [45]:
target_n <- 1000

data("sim_trajectory", package = "miloR")
## Extract SingleCellExperiment object
traj_sce <- sim_trajectory[['SCE']]
colData(traj_sce) <- DataFrame(sim_trajectory[["meta"]])
colnames(traj_sce) <- colData(traj_sce)$cell_id
redim <- reducedDim(traj_sce, "PCA")
dimnames(redim) <- list(colnames(traj_sce), paste0("PC", c(1:50)))
reducedDim(traj_sce, "PCA") <- redim 

# Upsample
new_indices <- sample(seq_len(ncol(traj_sce)), size = target_n, replace = TRUE)
traj_sce_upsampled <- traj_sce[, new_indices]

# Run Milo
traj_milo <- buildGraph(traj_milo, k=150, d=10, reduced.dim="PCA")
traj_milo <- makeNhoods(traj_milo, k=15, d=10, refined=TRUE, prop=0.01, refinement_scheme="graph")
traj_milo <- countCells(traj_milo, meta.data = data.frame(colData(traj_milo)), samples="Sample")

Constructing kNN graph with k:150

Checking valid object

Running refined sampling with graph

Checking meta.data validity

Counting cells in neighbourhoods



In [47]:
traj_design <- data.frame(colData(traj_milo))[,c("Sample", "Condition")]
traj_design <- distinct(traj_design)
rownames(traj_design) <- traj_design$Sample
## Reorder rownames to match columns of nhoodCounts(milo)
traj_design <- traj_design[colnames(nhoodCounts(traj_milo)), , drop=FALSE]
milo_res <- testNhoods(
    traj_milo,
    design=~Condition,
    design.df=traj_design,
    reduced.dim="PCA",
    fdr.weighting="graph-overlap"
)

Using TMM normalisation

Running with model contrasts

Performing spatial FDR correction with graph-overlap weighting

